<a href="https://colab.research.google.com/github/Khoawawa/ArtDetective/blob/main/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

import kagglehub

# Download latest version
path = kagglehub.dataset_download("alessandrasala79/ai-vs-human-generated-dataset")

print("Path to dataset files:", path)

100%|██████████| 9.76G/9.76G [01:31<00:00, 114MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/alessandrasala79/ai-vs-human-generated-dataset/versions/4


In [ ]:
import os
import sys
import cv2
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from PIL import Image
from IPython.display import display
import seaborn as sns
import time
import random

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import AdamW, Adam
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as T
from torchvision.transforms import InterpolationMode
import torchvision.models as models
from sklearn.model_selection import train_test_split
import random

In [ ]:
def set_seed(seed=42):
    torch.manual_seed(seed)  # For CPU
    torch.cuda.manual_seed(seed)  # For GPU
    torch.cuda.manual_seed_all(seed)  # If using multi-GPU
    np.random.seed(seed)  # For NumPy
    random.seed(seed)  # For Python's random module
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
set_seed()

In [ ]:
# path = "/content/dataset" if "google.colab" in sys.modules else "E:/Code/Python/ML/dataset/HumanvsAi"
TRAIN_CSV = path + '/train.csv'
TEST_CSV = path + '/test.csv'
DATA_DIR = path

In [ ]:
train_df = pd.read_csv(TRAIN_CSV)
train_df = train_df[['file_name','label']]
train_df.columns = ['id','label']

In [ ]:
test_df = pd.read_csv(TEST_CSV)
test_df.head()

,id
0,test_data_v2/1a2d9fd3e21b4266aea1f66b30aed157.jpg
1,test_data_v2/ab5df8f441fe4fbf9dc9c6baae699dc7.jpg
2,test_data_v2/eb364dd2dfe34feda0e52466b7ce7956.jpg
3,test_data_v2/f76c2580e9644d85a741a42c6f6b39c0.jpg
4,test_data_v2/a16495c578b7494683805484ca27cf9f.jpg


In [ ]:
IMG_SIZE = (224,224)

In [ ]:
train_transform = T.Compose([
    T.Resize(IMG_SIZE, interpolation=InterpolationMode.BICUBIC),
    T.RandomHorizontalFlip(),
    T.RandomVerticalFlip(),
    T.RandomRotation(20),
    T.GaussianBlur(kernel_size=(7, 13), sigma=(0.1, 1.0)),
    T.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [ ]:
test_transforms = T.Compose([
    T.Resize(IMG_SIZE, interpolation=InterpolationMode.BICUBIC),
    T.CenterCrop(IMG_SIZE),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [ ]:
train_size = len(train_df)

In [ ]:
df_train, df_test = train_test_split(train_df[:train_size], test_size=0.2, stratify=train_df.iloc[:train_size, 1],random_state=42)

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, dataframe, root_dir, transform=None):
        self.data = dataframe
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.data.iloc[idx, 0])  # Image filename
        image = Image.open(img_name).convert('RGB')
        label = int(self.data.iloc[idx, 1])  # Label

        if self.transform:
            image = self.transform(image)

        return image, label

In [ ]:
train_dataset = CustomDataset(df_train, DATA_DIR, transform=train_transform)
test_dataset = CustomDataset(df_test, DATA_DIR, transform=test_transforms)

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset,batch_size=32,shuffle=True)

In [ ]:
class CNN(nn.Module):
  def __init__(self,img_size=224,channels=3,classes=1):
    super(CNN, self).__init__()

    self.features = nn.Sequential(
      nn.Conv2d(in_channels=channels,out_channels=32,kernel_size=(3,3),stride=1,padding=1),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size=(2,2),stride=2),
      nn.Conv2d(in_channels=32,out_channels=64,kernel_size=(3,3),stride=2,padding=1),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size=(2,2),stride=2),
      nn.Conv2d(in_channels=64,out_channels=128,kernel_size=(3,3),stride=2,padding=1),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size=(2,2),stride=2)
    )
    with torch.no_grad():
            dummy_input = torch.randn(1, channels, img_size, img_size)
            output = self.features(dummy_input)
            linear_in = output.view(1, -1).shape[1]  # Tính số phần tử đầu vào
    self.classifier = nn.Sequential(
      nn.Linear(in_features=int(linear_in),out_features=classes)
    )
  def _calculate_output_size(self,W,K,P,S):
    return 1 + (W - K + 2*P)/S
  def forward(self, x):
    x = self.features(x)

    x = torch.flatten(x, start_dim=1)

    x = self.classifier(x)
    return x


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
model = models.resnet50(pretrained=True)
ftrs = model.fc.in_features
model.fc = nn.Linear(ftrs, 2)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 153MB/s]


In [ ]:
# model definition
model_path = '/content/drive/My Drive/ML/modelV1_n.pth'
model = CNN()
model.load_state_dict(torch.load(model_path))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
# epoch
num_epochs = 3
# optimizer and loss function
optimizer = AdamW(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/My Drive/ML/modelV1_n.pth'

In [ ]:
def train(model, train_loader, criterion, optimizer, n_epochs):
    model.train()
    for epoch in range(n_epochs):
        start_time = time.time()

        total_loss = 0
        correct = 0
        total = 0

        for imgs, labels in train_loader:
            imgs, labels = imgs.to(device), labels.to(device).long()  # Ensure labels are integers

            pred = model(imgs)  # Logits from ResNet
            loss = criterion(pred, labels)  # CrossEntropyLoss expects (logits, class indices)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

            predicted = torch.argmax(pred, dim=1)  # Get class index
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

        acc = correct / total * 100
        avg_loss = total_loss / len(train_loader) if len(train_loader) > 0 else 0
        epoch_time = time.time() - start_time

        print(f"Epoch {epoch+1}/{n_epochs}, Loss: {avg_loss:.4f}, Accuracy: {acc:.2f}%, Time: {epoch_time:.2f}s")

train(model, train_loader, criterion, optimizer, num_epochs)


KeyboardInterrupt: 

In [ ]:
# testing model
model.eval()

correct = 0
total = 0

# Disable gradient calculation for evaluation
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        _, predicted = torch.max(outputs, 1)

        correct += (predicted == labels).sum().item()
        total += labels.size(0)

accuracy = correct / total
print(f"Test Accuracy: {accuracy:.4f}")

In [ ]:
save_path = "/content/drive/My Drive/ML/resnetV1_3.pth"
torch.save(model.state_dict(), save_path)